In [66]:
#-*- coding: latin1 -*-
import numpy as np
import pandas as pd
import os
from matplotlib import pyplot as plt

from keras.datasets import cifar10
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.layers import Dropout
from keras.layers import BatchNormalization

plt.rcParams['figure.figsize'] = [15, 10]

# Description

- CIFAR-10 jest zbiorem zawierającym 60.000 zdjęć, każde w rozmiarach 32x32 pikseli. Zdjęcia można podzielić na 10 klas (6000 zdjęć każdej klasy). Zbiór uczący składa się z 50.000 instancji, zbiór testowy z 10.000.
- Instancje podzielone są na 5 batchy uczących i 1 testujący. Każdy batch zawiera 10.000 zdjęć.
- Pojedynczy batch zawiera 1000 losowo wybranych zdjęć z każdej klasy.
- Klasy zdjęć:  
  - airplane
  - automobile
  - bird
  - cat
  - deer
  - dog
  - frog
  - horse
  - ship
  - truck

Używany w dalszej części projektu termin "skuteczność klasyfikacji" oznacza stosunek poprawnie sklasyfikowanych obrazów zbioru testowego, w stosunku do całkowitej liczby elementów w tym zbiorze pomnożony przez 100%. Przykładowo "skuteczność 80%" oznacza, że w 80%-tach przypadków sieć poprawnie sklasyfikowała wybrany obraz znajdujący się w zbiorze testowym.

Używany w dalszej części projektu termin "epoka" lub "iteracja" oznacza proces przetworzenia wszystkich elementów zbioru uczącego.

# Helper functions

In [67]:
# load train and test dataset
def load_dataset():
    # load dataset
    (trainX, trainY), (testX, testY) = cifar10.load_data()
    
    # one hot encode target values (transform integer into a 10 element binary vector with a a1 for the class index of the value)
    trainY = np_utils.to_categorical(trainY)
    testY = np_utils.to_categorical(testY)
    return trainX, trainY, testX, testY


# scale pixels
def scale_pixels(train, test):
    # convert: integers -> float32
    train_norm = train.astype('float32')
    test_norm = test.astype('float32')
    
    # normalize to range 0-1
    train_norm = train_norm / 255.0
    test_norm = test_norm / 255.0
    
    # return normalized images
    return train_norm, test_norm


# plot diagnostic learning curves
def show_summary(history):
    # plot loss
    plt.subplot(211)
    plt.title('Cross Entropy Loss')
    plt.plot(history.history['loss'], color='blue', label='train')
    plt.plot(history.history['val_loss'], color='orange', label='test')
    # plot accuracy
    plt.subplot(212)
    plt.title('Classification Accuracy')
    plt.plot(history.history['acc'], color='blue', label='train')
    plt.plot(history.history['val_acc'], color='orange', label='test')

    
# run test
def run_test(mod, iterations = None):
    # load dataset
    trainX, trainY, testX, testY = load_dataset()
    
    # scale pixels
    trainX, testX = scale_pixels(trainX, testX)
    
    if iterations is None:
        iterations = 100
    
    # fit model
    history = mod.fit(trainX, trainY, 
                        epochs = iterations, 
                        batch_size = 64, 
                        validation_data = (testX, testY), 
                        verbose = 1)

    # evaluate model
    _, acc = mod.evaluate(testX, testY, verbose = 0)

    # print accuracy
    print('Accuracy (on testing set): > %.3f' % (acc * 100.0))
    
    # return history
    return history

# Load the dataset

In [ ]:
# load the dataset
trainX, trainY, testX, testY = load_dataset()

# dataset summary
print('Training data: X = %s, y = %s' % (trainX.shape, trainY.shape))
print('Testing data: X = %s, y = %s' % (testX.shape, testY.shape))

# Show example images

In [ ]:
# plot sample images
for i in range(9):
    # define subplot
    plt.subplot(330 + 1 + i)
    # plot raw pixel data
    plt.imshow(trainX[i])
plt.show()

# Prepare pixel data

In [ ]:
trainX, trainY = scale_pixels(trainX, trainY)

# Define base model (1 VGG block)

Na początku zdefiniowano podstawowy model VGG, który składa się z:
- (1) dwóch warstw konwolucyjnych o rozmiarach 3x3,
- (2) jednej warstwy dokonującej max-pooling,
- (3) dwóch warstw w pełni połączonych

Warstwy (1) i (2) tworzą pewnego rodzaju "blok", który może być powielany, przy czym liczba filtrów w każdym bloku będzie wzrastała dwukrotnie wraz z każdym kolejnym blokiem.

Padding został użyty na warstwach konwolucyjnych po to, by upewnić się, że rozmiary warstw wyjściowych będą takie same jak rozmiary warstwy wejściowej.

Użytym algorytmem optymalizującym był SGD (Stochastic Gradient Descent)

In [ ]:
# define cnn model
def define_model_v1():
    # create sequential model
    model = Sequential()
    
    # add convolution
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    
    # add pooling
    model.add(MaxPooling2D((2, 2)))
    
    # flatten (flattens input into a single vector)
    model.add(Flatten())
    
    # fully connected layer (128 units, ReLU activation function)
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    
    # fully connected layer (10 units, softmax activation function)
    model.add(Dense(10, activation='softmax'))
    
    # compile model
    opt = SGD(lr = 0.001, momentum = 0.9)
    model.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
    return model

model = define_model_v1()

# run test
history = run_test(model)

In [ ]:
# show summary
show_summary(history)

Jak widać powyżej, sieć neuronowa oparta o model zbudowany z jednego bloku VGG przeprowadziła poprawną klasyfikację obrazu w 66.6%. Jest to dobry wynik, zważając na to, że zbiór zawiera aż 10 różnych klas, więc prawdopodobieństwo trafienia "na ślepo" wynosi 10%.

Jak widać na powyższych wykresach, dość szybko następuje zjawisko przeuczenia i już po 15 iteracjach wartość funkcji straty dla danych testowych zaczyna gwałtownie rosnąć, pomimo że skuteczność klasyfikacji sieci dla danych ze zbioru uczącego wynosi w tym czasie dopiero ok. 85%.

Najlepsze wyniki dla tego zbioru danych uzyskiwane przy użyciu konwolucyjnych sieci neuronowych wynosiły ponad 90%. Jest to wynik znacznie większy, niż ten uzyskany powyżej, dlatego w kolejnych etapach projektu zwiększono liczbę bloków VGG i zbadano ich wpływ na skuteczność klasyfikacji.

# Define modified model (with 2 VGG blocks)

W kolejnej części projektu zmodyfikowano poprzedni model, dodając do niego kolejny blok VGG i porównano wyniki. Jak wspomnianio wcześniej, podczas dokładania kolejnych bloków VGG, liczba zastosowanych filtrów będzie zwiększana dwukrotnie z każdym dodanym blokiem.

In [ ]:
# define cnn model
def define_model_v2():
    # create sequential model
    model = Sequential()
    
    # add convolution (1st VGG block)
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    
    # add pooling
    model.add(MaxPooling2D((2, 2)))
    
    # add convolution (2st VGG block)
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    
    # add pooling
    model.add(MaxPooling2D((2, 2)))
    
    # flatten (flattens input into a single vector)
    model.add(Flatten())
    
    # fully connected layer (128 units, ReLU activation function)
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    
    # fully connected layer (10 units, softmax activation function)
    model.add(Dense(10, activation='softmax'))
    
    # compile model
    opt = SGD(lr = 0.001, momentum = 0.9)
    model.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
    return model

model = define_model_v2()

# run test
history = run_test(model)

In [ ]:
# show summary
show_summary(history)

Jak widać powyżej, rozbudowanie modelu o jeden blok VGG nieznacznie zwiększył skuteczność sieci (do ok. 71%), jednak nadal dosyć szybko pojawia się zjawisko przeuczenia.

W kolejnym etapie do sieci dodano jeszcze jeden blok i zbadano, czy skuteczność wykrywania ponownie się polepszy.

# Define modified model (with 3 VGG blocks)

In [ ]:
# define cnn model
def define_model_v3():
    # create sequential model
    model = Sequential()
    
    # add convolution (1st VGG block)
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    
    # add pooling
    model.add(MaxPooling2D((2, 2)))
    
    # add convolution (2st VGG block)
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    
    # add pooling
    model.add(MaxPooling2D((2, 2)))
    
    # add convolution (3st VGG block)
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    
    # add pooling
    model.add(MaxPooling2D((2, 2)))
    
    # flatten (flattens input into a single vector)
    model.add(Flatten())
    
    # fully connected layer (128 units, ReLU activation function)
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    
    # fully connected layer (10 units, softmax activation function)
    model.add(Dense(10, activation='softmax'))
    
    # compile model
    opt = SGD(lr = 0.001, momentum = 0.9)
    model.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
    return model

model = define_model_v3()

# run test
history = run_test(model)

In [ ]:
# show summary
show_summary(history)

Skuteczność sieci z trzema blokami VGG wynosi ok. 74%, jednak ciągle widoczne jest tu zjawisko przeuczenia, które występuje już po niecałych 20-tu iteracjach.

# Define improved model (with 3 VGG + dropout)

W kolejnym etapie projektu dodano do poprzedniej sieci "Dropout", który polega na usuwaniu pewnych połączeń pomiędzy neuronami w sieci, z pewnym prawdopodobieństwem. Zabieg ten ma na celu obniżenie wrażliwości modelu na przeuczenie. Jest to forma regularyzacji modelu. Dzięki zastosowaniu warstwy "Dropout", sieć będzie starała się dostrzegać niezależne cechy i brak jednej z nich nie będzie wtedy problemem. Powinno to znacznie obniżyć podatność sieci na przeuczenie.

Model z punktu wyżej zmodyfikowano poprzez dodanie warstw "Dropout" po każdym poolingu oraz po warstwie w pełni połączonej. "Dropout rate" ustawiony został na 20% (usuwane jest 20% połączeń).

In [ ]:
# define cnn model
def define_model_v3_dropout():
    # create sequential model
    model = Sequential()
    
    # add convolution (1st VGG block)
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    
    # add pooling
    model.add(MaxPooling2D((2, 2)))
    
    # add dropout
    model.add(Dropout(0.2))
    
    # add convolution (2st VGG block)
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    
    # add pooling
    model.add(MaxPooling2D((2, 2)))
    
    # add dropout
    model.add(Dropout(0.2))
    
    # add convolution (3st VGG block)
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    
    # add pooling
    model.add(MaxPooling2D((2, 2)))
    
    # add dropout
    model.add(Dropout(0.2))
    
    # flatten (flattens input into a single vector)
    model.add(Flatten())
    
    # fully connected layer (128 units, ReLU activation function)
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    
    # add dropout
    model.add(Dropout(0.2))
    
    # fully connected layer (10 units, softmax activation function)
    model.add(Dense(10, activation='softmax'))
    
    # compile model
    opt = SGD(lr = 0.001, momentum = 0.9)
    model.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
    return model

model = define_model_v3_dropout()

# run test
history = run_test(model)

In [ ]:
# show summary
show_summary(history)

Jak widać powyżej, dodanie "Dropout"'ów znacznie poprawiło skuteczność sieci dla danych testowych (ok. 83%), a zjawisko przeuczenia sieci jest już praktycznie niewidoczne. 

# Define improved model (with 3 VGG + dropout + batch normalization)

Przy każdym poprzednim modelu, sieć trenowana była w 100 iteracjach (epokach). W następnym etapie projektu zwiększono tę liczbę, by sprawdzić czy model będzie w stanie jeszcze bardziej zwiększyć swoją dokładność.

W poprzednim punkcie zlikwidowane zostało zjawisko overfittingu i wartość funkcji straty nie rośnie nawet w setnej epoce, dlatego można bezpiecznie zwiększyć ilość iteracji i sprawdzić jak zachowa się model.

W celu przyspieszenia procesu nauki przy zwiększonej liczbie epok, zastosowano normalizację batch'u. Normalizacja batch'u jest techniką, która przyspiesza proces nauki i zwiększa jej stabilność. Używana jest to normalizacji warstwy wejściowej poprzez regulację i skalowanie funkcji aktywacji.

W celu zwiększenia regularyzacji, zwiększano również stopiniowo wartość "Dropout"'u w każdej warstwie.

Liczbę epok zwiększono do 400.

In [ ]:
# define cnn model
def define_model_v3_dropout_normalization():
    # create sequential model
    model = Sequential()
    
    # add convolution (1st VGG block)
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    
    # add pooling
    model.add(MaxPooling2D((2, 2)))
    
    # add dropout
    model.add(Dropout(0.2))
    
    # add convolution (2st VGG block)
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    
    # add pooling
    model.add(MaxPooling2D((2, 2)))
    
    # add dropout
    model.add(Dropout(0.3))
    
    # add convolution (3st VGG block)
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    
    # add pooling
    model.add(MaxPooling2D((2, 2)))
    
    # add dropout
    model.add(Dropout(0.4))
    
    # flatten (flattens input into a single vector)
    model.add(Flatten())
    
    # fully connected layer (128 units, ReLU activation function)
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    
    # add dropout
    model.add(Dropout(0.5))
    
    # fully connected layer (10 units, softmax activation function)
    model.add(Dense(10, activation='softmax'))
    model.add(BatchNormalization())
    
    # compile model
    opt = SGD(lr = 0.001, momentum = 0.9)
    model.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
    return model

model = define_model_v3_dropout_normalization()

# run test
history = run_test(model, 100)

Train on 50000 samples, validate on 10000 samples
Epoch 1/100
50000/50000 [==============================] - 14s 278us/step - loss: 8.9506 - acc: 0.1508 - val_loss: 11.4392 - val_acc: 0.1141
Epoch 2/100
50000/50000 [==============================] - 11s 229us/step - loss: 10.7016 - acc: 0.1111 - val_loss: 11.3754 - val_acc: 0.1037
Epoch 3/100
50000/50000 [==============================] - 11s 230us/step - loss: 11.0724 - acc: 0.0971 - val_loss: 11.0436 - val_acc: 0.0860
Epoch 4/100
50000/50000 [==============================] - 11s 228us/step - loss: 10.7972 - acc: 0.0857 - val_loss: 10.6398 - val_acc: 0.0761
Epoch 5/100
50000/50000 [==============================] - 12s 232us/step - loss: 9.1851 - acc: 0.0831 - val_loss: 6.7262 - val_acc: 0.0807
Epoch 6/100
50000/50000 [==============================] - 12s 230us/step - loss: 6.8007 - acc: 0.0854 - val_loss: 6.6442 - val_acc: 0.0990
Epoch 7/100
50000/50000 [==============================] - 11s 229us/step - loss: 6.8661 - acc: 0.0981 

In [ ]:
# show summary
show_summary(history)